### Collection

In [ ]:

import os
from src.StreamPort.device.analyses import MassSpecAnalyses

#path = "C:/Users/Sandeep/Desktop/Error-LC/Method-Data"
batches = os.listdir(path)
batches = [os.path.join(path, file) for file in batches]

error_lc_files = []
for batch in batches:
    batch_files = os.listdir(batch)
    batch_files = [os.path.join(batch, file) for file in batch_files if ".D" in file]
    error_lc_files.extend(batch_files)

analyses = MassSpecAnalyses(files=error_lc_files)  
print("Number of analyses: ", len(analyses.data))


No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250613_Mix-1_10x100ng-mL 2025-06-13 17-02-43\016-NV-Standby.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250613_Mix-1_10x10ng-mL 2025-06-13 14-40-12\016-NV-Standby.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250616_Mix-1_50x100ng-mL 2025-06-16 10-06-17\064-NV-StandBy.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250617_Mix-1_50x10ng-mL 2025-06-17 09-19-10\064-NV-StandBy.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250620_Mix-1_25x100ng-mL 2025-06-20 09-26-34\034-NV-StandBy.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250623_Mix-1_25x100ng-mL 2025-06-23 15-57-58\034-NV-StandBy.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250624_Mix-1_20x100ng-mL 2025-06-24 16-17-42\028-NV-StandBy.D
No MS2 data found in d

mzs available in SIM: 120, 124, 134, 237, 247, 254, 258, 267, 268, 275, 278, 296, 300, 325, 370, 429, 441, 748, 752

mzs available in TIC: range(100, 801)

In [3]:
# EIC/TIC plots (intensity/time)
analyses.plot_chromatogram(indices = [0, 1, 2, 3, 4, 5], data="sim", mz=268)

In [ ]:
# Base Peak Chromatogram (intensity/time)
analyses.plot_bpc(indices = 8)

In [ ]:
# Mass Spectrum (intensity/mz)
analyses.plot_ms(indices = 0) 

In [ ]:
# 3D plots (mz/rt/intensity)
analyses.plot_3d(indices = 0)

### Feature Extraction

In [ ]:

from src.StreamPort.device.methods import MassSpecMethodExtractFeaturesNative
#processor = MassSpecMethodExtractFeaturesNative(data="tic", rt=2.233, mz=236, rt_window_size=6, mz_window_size=1.0, smooth=0, exclude = 2(see docs)) # Disabling smoothing improves fit quality 
processor = MassSpecMethodExtractFeaturesNative(data="tic", rt=1.94, mz=268, rt_window_size=10, mz_window_size=1.0, smooth=False, exclude=0) # highly dependent on correct choice of target
analyses = processor.run(analyses) # Fit debug values show a 2-class spread of amplitudes.


Flush found at index 0, Skipping extraction...
Flush found at index 1, Skipping extraction...
Sample index 2 Warning: Peak near window edge at rt:1.89155, mz:102.0
Sample index 8 Warning: Peak near window edge at rt:1.9064833333333333, mz:102.0
Sample index 14 Warning: Peak near window edge at rt:1.8766166666666666, mz:102.0
Flush found at index 15, Skipping extraction...
Flush found at index 16, Skipping extraction...
Sample index 17 Warning: Peak near window edge at rt:1.8617, mz:102.0
Sample index 23 Warning: Peak near window edge at rt:2.0855333333333332, mz:100.0
Sample index 29 Warning: Peak near window edge at rt:1.9064666666666668, mz:102.0
Flush found at index 30, Skipping extraction...
Flush found at index 31, Skipping extraction...
Sample index 32 Warning: Peak near window edge at rt:1.8760166666666667, mz:102.0
Sample index 38 Warning: Peak near window edge at rt:1.8762666666666667, mz:102.0
Sample index 44 Warning: Peak near window edge at rt:1.93585, mz:102.0
Sample index

In [4]:
gauss_fit_plots = analyses.plot_gaussian_fit(indices=[0, 1, 2, 3]) # the same indices passed are the keys to the respective plots in the returned dict

No features have been extracted for sample 0
No features have been extracted for sample 1


In [5]:

for i in gauss_fit_plots.keys():
    if not isinstance(gauss_fit_plots[i], type(None)):
        gauss_fit_plots[i].show()

In [6]:
features = analyses.get_features()
features.describe()

None type features found for sample 0. Skipping...
None type features found for sample 1. Skipping...
None type features found for sample 15. Skipping...
None type features found for sample 16. Skipping...
None type features found for sample 30. Skipping...
None type features found for sample 31. Skipping...
None type features found for sample 93. Skipping...
None type features found for sample 94. Skipping...
None type features found for sample 156. Skipping...
None type features found for sample 157. Skipping...
None type features found for sample 189. Skipping...
None type features found for sample 190. Skipping...
None type features found for sample 222. Skipping...
None type features found for sample 223. Skipping...
None type features found for sample 261. Skipping...
None type features found for sample 262. Skipping...
None type features found for sample 288. Skipping...
None type features found for sample 289. Skipping...
None type features found for sample 321. Skipping...
Non

,peak_height,peak_rt,peak_mz,peak_area_2d,peak_volume,peak_fit_2d,peak_v/a_error,peak_s/n_2d,peak_mean,peak_fwhm_rt,peak_s/n,peak_fit_quality_rt
count,3.950000e+02,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,7.544005e+05,1.932501,268.030380,54089.044775,68880.476306,5586.724036,1.733951,194.775011,59566.891254,0.078501,3.882992,-0.445284
std,5.494098e+05,0.036325,0.375775,39248.707230,49921.737042,3619.349493,1.341353,112.517022,42890.164461,0.023215,0.871842,0.607596
min,1.341000e+03,1.772600,267.000000,273.278254,400.591196,269.828305,0.791191,1.546194,436.287879,0.014917,3.378416,-6.653637
25%,1.845440e+05,1.921300,268.000000,13707.218202,14668.281114,1611.690491,1.188809,151.253887,15393.462121,0.074567,3.547779,-0.455304
50%,1.023168e+06,1.921633,268.000000,72229.042179,91298.935453,7156.478943,1.307755,243.065414,79318.893939,0.074567,3.615348,-0.376633
75%,1.106944e+06,1.936483,268.000000,78594.594117,103696.276552,8169.840955,1.329726,279.584693,86116.765152,0.074583,3.724537,-0.247357
max,1.537536e+06,2.085767,269.000000,110248.425471,144566.886322,11590.502446,10.400553,351.422594,120756.727273,0.313217,11.838432,0.307218


In [7]:
analyses.plot_features() # plots by default features where no None or NA values are present 

### Train Set Selection

In [ ]:

from datetime import datetime

all_batches = analyses.get_batches()
date_threshold = datetime(2025, 6, 20)
train_batch_names = [batch for batch in all_batches if datetime.strptime(" ".join(batch.split(" ")[-2:]), "%Y-%m-%d %H-%M-%S") < date_threshold]

"""
Using the same train batch from PressureCurvesAnalyses
"""
train_indices = []
for batch in train_batch_names:
    indices = sorted(analyses.get_batch_indices(batch))

    metadata = analyses.get_metadata(indices=indices)
    metadata.sort_values("index", inplace=True)
    
    # Filter out additional data
    metadata = metadata[
        (metadata["batch_position"] > 4) & # if batch position above 4 
        (~metadata["sample"].isin(["Flush", "Blank"])) & # if "Flush" or "Blank" 
        (metadata["batch"].str.contains("x100ng-mL", na=False)) # if Mix 100ng-mL
        ]

    indices = sorted(metadata["index"])
    train_indices.extend(indices)

for i in [4, 5, 6, 124, 125]:
    if i in train_indices:
        train_indices.remove(i)
train_indices.sort()
print("Train indices: ", len(train_indices), " ", train_indices)


Train indices:  112   [7, 9, 10, 11, 12, 13, 19, 20, 21, 22, 24, 25, 26, 27, 28, 34, 35, 36, 37, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 97, 98, 99, 100, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 114, 115, 116, 117, 118, 120, 121, 122, 123, 126, 127, 128, 129, 130, 132, 133, 134, 135, 136, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154]


In [ ]:
analyses.plot_features(indices=train_indices) # train set only 50x100ng/ml

In [11]:
analyses.plot_chromatogram(indices=[91, 154], mz=268)

### Scaling

In [ ]:

from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

metadata = analyses.get_metadata(train_indices)
variables = analyses.get_features(train_indices)

ml = MachineLearningAnalyses(variables, metadata)

from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scaler = MachineLearningScaleFeaturesScalerSklearn("StandardScaler")
ml = scaler.run(ml)


In [14]:
ml.plot_data()

### Machine Learning

In [ ]:

from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn

iso = MachineLearningMethodIsolationForestSklearn()
ml = iso.run(ml)
ml.train()


In [16]:
ml.train()
ml.plot_scores(threshold="min")

In [ ]:
remaining_batches = list(set(all_batches) - set(train_batch_names))
remaining_batches.sort(key=lambda f: (
    datetime.strptime(f.split(' ')[-2] + '_' + f.split(' ')[-1], "%Y-%m-%d_%H-%M-%S")  # Sort by date
    )  
)

In [18]:
tests = {}
date_threshold_old = date_threshold
for batch in remaining_batches:
    batch_date = datetime.strptime(" ".join(batch.split(" ")[-2:]), "%Y-%m-%d %H-%M-%S") 

    if batch_date >= date_threshold: # search and collect test samples
        date_threshold_old = batch_date 
        
        test_indices = analyses.get_batch_indices(batch)
        if 3 in test_indices:
            #print(analyses.get_metadata(3))
            test_indices.remove(3)

        test_indices_copy = test_indices
        for j in test_indices:
            ft = analyses.get_features(j)
            if ft.empty:
                continue
            mt = analyses.get_metadata(j)
            tests[f"{batch_date.strftime('%Y-%m-%d_%H-%M-%S')}_index_{j}"] = (ft, mt)

None type features found for sample 156. Skipping...
None type features found for sample 157. Skipping...
None type features found for sample 189. Skipping...
None type features found for sample 190. Skipping...
None type features found for sample 222. Skipping...
None type features found for sample 223. Skipping...
None type features found for sample 261. Skipping...
None type features found for sample 262. Skipping...
None type features found for sample 288. Skipping...
None type features found for sample 289. Skipping...
None type features found for sample 321. Skipping...
None type features found for sample 322. Skipping...
None type features found for sample 366. Skipping...
None type features found for sample 367. Skipping...
None type features found for sample 417. Skipping...
None type features found for sample 418. Skipping...
None type features found for sample 419. Skipping...
None type features found for sample 420. Skipping...
None type features found for sample 421. Skipp

In [19]:
dates = list(tests.keys())
print("Number of test samples: ", len(dates))

Number of test samples:  247


### Testing

In [21]:
import pandas as pd
for i in range(1, 20):
    n = i
    date = dates[n]
    test_data = tests[date][0]
    test_metadata = tests[date][1]

    print(f"Test {date}: index:", tests[date])

    ml.predict(test_data, test_metadata)

    outliers = ml.test_prediction_outliers(threshold="min", show_scores=False) # defaults: n_tests = _get_num_tests(), show_scores = False

    #fig_test_scores = ml.plot_scores(threshold="min")
    #fig_test_scores.update_layout(title=f"Test set {date} final run")
    #fig_test_scores.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_scores.png", width=1100, height= 350, scale = 3)
    #fig_test_scores.show()


    # confidence_plot = ml.plot_confidence_variation()
    # confidence_plot.show()
    # #confidence_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_confidence_variation.png", width=1100, height= 350, scale = 3)

    # threshold_plot = ml.plot_threshold_variation()
    # threshold_plot.show()
    # #threshold_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_threshold_variation_serialized_tests.png", width=1100, height= 350, scale = 3)

    # train_time_plot = ml.plot_train_time()
    # train_time_plot.show()
    # #train_time_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_train_time_variation.png", width=1100, height= 350, scale = 3)

    # stability_plot = ml.plot_model_stability()
    # stability_plot.show()
    # #stability_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_model_stability_vs_train_size.png", width=1100, height= 350, scale = 3)


    data = pd.concat([test_metadata, test_data], axis=1)

    if outliers["outliers"]["class"].iloc[0] == "outlier":
        outlier_data = data

        if os.path.exists("dev/msd_outliers.csv"):
            outliers_file = pd.read_csv("dev/msd_outliers.csv")
            outliers_file = pd.concat([outliers_file, outlier_data], axis = 0)
            outliers_file.drop_duplicates(subset="index", inplace = True, ignore_index=True)
            outliers_file.to_csv("dev/msd_outliers.csv", index=False)
        
        else:
            outlier_data.to_csv("dev/msd_outliers.csv", index=False)

    else:

        if os.path.exists("dev/msd_normals.csv"):
            normals_file = pd.read_csv("dev/msd_normals.csv")

            normals_file = pd.concat([normals_file, data], axis = 0)
            normals_file.drop_duplicates(subset="index", inplace=True, ignore_index=True)
            normals_file.to_csv("dev/msd_normals.csv", index=False)
        
        else:
            data.to_csv("dev/msd_normals.csv", index=False)

    ml.add_prediction() # add_data already calls self.train()


Test 2025-06-20_09-26-34_index_159: index: (   peak_height   peak_rt  peak_mz  peak_area_2d   peak_volume  peak_fit_2d  \
0      1228288  1.921367    268.0  89546.932608  118002.44064   9829.96274   

   peak_v/a_error  peak_s/n_2d     peak_mean  peak_fwhm_rt  peak_s/n  \
0        1.317772   252.316853  98495.121212      0.074567  3.589568   

   peak_fit_quality_rt  
0            -0.354485  ,    index              name                                               path  \
0    159  004-D2F-A1-Mix 1  C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   

                                          batch  batch_position  idle_time  \
0  250620_Mix-1_25x100ng-mL 2025-06-20 09-26-34               4       44.0   

  sample    method           timestamp  unit instrument signal  \
0  Mix 1  error_lc 2025-06-20 09:53:26  None       None   None   

                        date detector    pump vial_position  \
0  20 Jun 25   9:54 am +0100       MS  G7116B       D2F-A1>   

           start_time  

In [ ]:
fig_test_scores = ml.plot_scores(threshold="min")
#fig_test_scores.update_layout(title=f"Test set {date} final run")
#fig_test_scores.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_scores.png", width=1100, height= 350, scale = 3)
fig_test_scores.show()


confidence_plot = ml.plot_confidence_variation()
confidence_plot.show()
# #confidence_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_confidence_variation.png", width=1100, height= 350, scale = 3)

threshold_plot = ml.plot_threshold_variation()
threshold_plot.show()
# #threshold_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_threshold_variation_serialized_tests.png", width=1100, height= 350, scale = 3)

train_time_plot = ml.plot_train_time()
train_time_plot.show()
# #train_time_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_train_time_variation.png", width=1100, height= 350, scale = 3)

stability_plot = ml.plot_model_stability()
stability_plot.show()
# #stability_plot.write_image(f"dev/figures/gif_figs/fig_error_lc_test_{date}_model_stability_vs_train_size.png", width=1100, height= 350, scale = 3)

In [23]:
ml.get_results(summarize = True)

Index: 158
Train size: 112
Num. Tests: 26
Normal: 0
Outlier: 26

Index: 159
Train size: 112
Num. Tests: 26
Normal: 23
Outlier: 3

Index: 160
Train size: 113
Num. Tests: 26
Normal: 26
Outlier: 0

Index: 161
Train size: 114
Num. Tests: 26
Normal: 26
Outlier: 0

Index: 162
Train size: 115
Num. Tests: 26
Normal: 26
Outlier: 0

Index: 163
Train size: 116
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 164
Train size: 117
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 165
Train size: 117
Num. Tests: 25
Normal: 24
Outlier: 1

Index: 166
Train size: 118
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 167
Train size: 119
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 168
Train size: 120
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 169
Train size: 121
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 170
Train size: 122
Num. Tests: 25
Normal: 0
Outlier: 25

Index: 171
Train size: 122
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 172
Train size: 123
Num. Tests: 25
Normal: 25
Outlier: 0

Index: 173
Train size: 12

In [24]:
if os.path.exists("dev/msd_outliers.csv"):
    msd_outliers = pd.read_csv("dev/msd_outliers.csv")
if os.path.exists("dev/error_lc_outliers.csv"):
    pc_outliers = pd.read_csv("dev/error_lc_outliers.csv")

pc_no_duplicates = pc_outliers.drop(columns = [col for col in pc_outliers.columns if col in msd_outliers.columns and col != "index"])

true_outliers = msd_outliers.merge(pc_no_duplicates, on = "index", how="inner")
true_outliers.drop_duplicates(subset="index")
print(true_outliers)

    index              name  \
0     158  003-D2F-B2-Blank   
1     164  009-D2F-B2-Blank   
2     170  015-D2F-B2-Blank   
3     184  029-D2F-A1-Mix 1   
4     185  030-D2F-A1-Mix 1   
..    ...               ...   
70    353  033-D2F-B1-Blank   
71    358  038-D2F-A1-Mix 1   
72    359  039-D2F-B1-Blank   
73    365  045-D2F-B1-Blank   
74    368  003-D2F-B1-Blank   

                                                 path  \
0   C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   
1   C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   
2   C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   
3   C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   
4   C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   
..                                                ...   
70  C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   
71  C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   
72  C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   
73  C:/Users/Sandeep/Desktop/Error-LC/Method-Data\...   


In [25]:
print(true_outliers["index"])

0     158
1     164
2     170
3     184
4     185
     ... 
70    353
71    358
72    359
73    365
74    368
Name: index, Length: 75, dtype: int64


### Data & Method Diagnostics 

In [ ]:
import pandas as pd 
ana = analyses.data[24]
print("Name: ", ana["name"], "\nSIM-data: ", pd.DataFrame({"mz" : ana["sim"]["mz"],"compound" : ana["sim"]["compound"], "internal_standard" : ana["sim"]["int_standard"]}))

In [ ]:
anas = analyses.data[:24]
for ana in anas:
    print(ana["sample"])

In [ ]:
data = ana["tic"]
print("Num_rts: ", len(data["rt"]))
print("Num_mzs: ", len(data["mz"]))
print("TIC mzs: ", data["mz"])

In [ ]:

rt = 0.05166667
mz = 237
if mz is not None:
    mz_index = data["mz"].tolist().index(mz) # find the index of the given mz
else:
    mz_index = 0
    mz = data["mz"][0]

if rt is not None:
    rt_array = data["rt"]
    # find closest value to given rt and plot it
    rt_index = abs(rt_array - rt).argmin() # returns index of closest rt to rt argument 
    y = (data["intensity"][:, mz_index]) * 0
    y[rt_index] = rt_array[rt_index]  
    # x = array([rt_array[rt_index]]                   
else:
    y = data["intensity"][:, mz_index] 
print("rt: ", rt_array, " ", rt_array.shape)
print("y: ", y, " ", y.shape)

### Rainbow Method Checks

In [ ]:

file = error_lc_files[2]
print(file)


import rainbow as rb
meta = rb.read_metadata(file)
print(meta)

datadir = rb.read(file)
print(datadir.metadata)


datafiles = datadir.datafiles
for f in datafiles: # xlabels= retention time, ylabels = mz/wavelength, data = intensity
    if f.name in ["MSD1.MS", "MSD2.MS"]:
        print(f.metadata)
        #print(f.get_info())

#print(f.extract_traces()) # returns data (intensity) for the given ylabels. Rows correspond to ylabels

f.plot(label=325) # label represents the ylabel (mz) for which data should be plotted